In [62]:
import opendatasets as od
import pandas
import os

od.download(
	"https://www.kaggle.com/datasets/arashnic/fitbit")


Skipping, found downloaded files in ".\fitbit" (use force=True to force download)


In [67]:
folder = './fitbit/Fitabase Data 4.12.16-5.12.16/'

#Import only the daily data (not houly or minute)
filenames = [folder + i for i in os.listdir(folder) if 'daily' in i.lower() or 'day' in i.lower()]
for index, name in enumerate(filenames):
    print(index, name)

0 ./fitbit/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv
1 ./fitbit/Fitabase Data 4.12.16-5.12.16/dailyCalories_merged.csv
2 ./fitbit/Fitabase Data 4.12.16-5.12.16/dailyIntensities_merged.csv
3 ./fitbit/Fitabase Data 4.12.16-5.12.16/dailySteps_merged.csv
4 ./fitbit/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv


In [68]:
# Importing the acitivity dataset
activity = pd.read_csv(filenames[0], parse_dates=['ActivityDate'])

# # Importing the sleep dataset
sleep = pd.read_csv(filenames[4])
sleep['SleepDay'] = pd.to_datetime(sleep['SleepDay'], format='%m/%d/%Y %I:%M:%S %p')
sleep.rename(columns={'SleepDay':'ActivityDate'}, inplace=True)

# # Importing the calories dataset
# calories = pd.read_csv(filenames[2], parse_dates=['ActivityDay'])
# calories.rename(columns={'ActivityDay':'ActivityDate'}, inplace=True)

# # Importing steps dataset
# steps = pd.read_csv(filenames[3], parse_dates=['ActivityDay'])
# steps.rename(columns={'ActivityDay':'ActivityDate', 'StepTotal':'TotalSteps'}, inplace=True)

# # Importing intensity dataset
# intensity = pd.read_csv(filenames[1], parse_dates=['ActivityDay'])
# intensity.rename(columns={'ActivityDay':'ActivityDate'}, inplace=True)

# print(activity.shape, sleep.shape, calories.shape, steps.shape, intensity.shape)
print(activity.shape, sleep.shape)

(940, 15) (413, 5)


In [71]:
import pandas as pd
activity.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,2016-04-12,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985
1,1503960366,2016-04-13,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797
2,1503960366,2016-04-14,10460,6.74,6.74,0.0,2.44,0.40,3.91,0.0,30,11,181,1218,1776
3,1503960366,2016-04-15,9762,6.28,6.28,0.0,2.14,1.26,2.83,0.0,29,34,209,726,1745
4,1503960366,2016-04-16,12669,8.16,8.16,0.0,2.71,0.41,5.04,0.0,36,10,221,773,1863


In [72]:
sleep.head()

,Id,ActivityDate,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12,1,327,346
1,1503960366,2016-04-13,2,384,407
2,1503960366,2016-04-15,1,412,442
3,1503960366,2016-04-16,2,340,367
4,1503960366,2016-04-17,1,700,712


In [37]:
print("We have data for ", len(fitbitData.Id.unique()), " different people")

We have data for  33  different people


In [74]:
#Combine activity and sleep dataframes
comb_df = activity.merge(sleep, how='outer', on=['Id', 'ActivityDate'])
comb_df.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985,1.0,327.0,346.0
1,1503960366,2016-04-13,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797,2.0,384.0,407.0
2,1503960366,2016-04-14,10460,6.74,6.74,0.0,2.44,0.40,3.91,0.0,30,11,181,1218,1776,NaN,NaN,NaN
3,1503960366,2016-04-15,9762,6.28,6.28,0.0,2.14,1.26,2.83,0.0,29,34,209,726,1745,1.0,412.0,442.0
4,1503960366,2016-04-16,12669,8.16,8.16,0.0,2.71,0.41,5.04,0.0,36,10,221,773,1863,2.0,340.0,367.0


In [105]:
#Check for duplicates
print("There are ",comb_df[comb_df.duplicated()].shape[0]," duplicates in the dataframe")
print("Removing duplicates...")
comb_df = comb_df.drop_duplicates().copy()

There are  3  duplicates in the dataframe
Removing duplicates...


In [126]:
#Check how many days people track data
act_dates = comb_df.groupby('Id')[('ActivityDate')].nunique().reset_index()
display(act_dates.ActivityDate.describe())
print("From our 33 users, most of them have logged activity for 31 different days. On average, people logged 28 days of activity with someone only logging 4, the minimum amount of days.")
print("A possible reason for no activity dates is that the user was not wearing the fitbit that day.")

count    33.000000
mean     28.484848
std       5.657524
min       4.000000
25%      29.000000
50%      31.000000
75%      31.000000
max      31.000000
Name: ActivityDate, dtype: float64

From our 33 users, most of them have logged activity for 31 different days. On average, people logged 28 days of activity with someone only logging 4, the minimum amount of days.
A possible reason for no activity dates is that the user was not wearing the fitbit that day.


In [140]:
#Check nulls
print("We have many dates without sleep records data")
print(comb_df.isnull().sum())

We have many dates without sleep records data
Id                            0
ActivityDate                  0
TotalSteps                    0
TotalDistance                 0
TrackerDistance               0
LoggedActivitiesDistance      0
VeryActiveDistance            0
ModeratelyActiveDistance      0
LightActiveDistance           0
SedentaryActiveDistance       0
VeryActiveMinutes             0
FairlyActiveMinutes           0
LightlyActiveMinutes          0
SedentaryMinutes              0
Calories                      0
TotalSleepRecords           530
TotalMinutesAsleep          530
TotalTimeInBed              530
dtype: int64


In [142]:
comb_df[comb_df.isnull()]

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
939,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
comb_df.isnull().sum()

Id                            0
ActivityDate                  0
TotalSteps                    0
TotalDistance                 0
TrackerDistance               0
LoggedActivitiesDistance      0
VeryActiveDistance            0
ModeratelyActiveDistance      0
LightActiveDistance           0
SedentaryActiveDistance       0
VeryActiveMinutes             0
FairlyActiveMinutes           0
LightlyActiveMinutes          0
SedentaryMinutes              0
Calories                      0
TotalSleepRecords           530
TotalMinutesAsleep          530
TotalTimeInBed              530
dtype: int64